<font face="B Mitra" size=4>
<div dir=rtl>
<p>


گراف محاسباتی گراف‌هایی جهت دار هستند که برای انجام عملیات‌های ریاضی تشکیل می‌شوند و در واقع عملیات ‌های ریاضی در آن شکسته می‌شود و برای محاسبات backward و forward در شبکه استفاده می‌شوند.
برای تشکیل گراف محاسباتی از کلاس Tensor استفاده می‌کنیم به طوری که کوچکترین ساختار موجود در پیاده سازی MLP تنسورها هستند و وزن‌ها و بایاس‌ها و ورودی‌های را از نوع تنسور تعریف می‌کنیم.

1. ایجاد تانسورها:
   - هر تانسور شامل این بخش‌ها است: مقدار (value)،  و اطلاعات مربوط به ورودی‌ها  (children ).
   - Value مقدار عددی Tensor.
  
   - children یک مجموعه از تنسور های فرزند، که معمولاً در عملیات ریاضی مورد استفاده قرار می‌گیرند که به عنوان ورودی به این تانسور داده می‌شوند. به این ترتیب، تانسورها می‌توانند به صورت سلسله‌مراتبی متصل شوند تا به یک گراف محاسباتی منتهی شوند.
   - با استفاده از magic methods عملگرهای ضرب و جمع و توان و تفریق را باز تعریف می‌کنیم به طوری که اعمال آنها بر روی تنسور معنا دار شود.
2. ایجاد گراف محاسباتی:
   - تنسورها به یکدیگر متصل می‌شوند و یک گراف محاسباتی را تشکیل می‌دهند.
   - هر اتصال بین تانسورها نمایش‌دهنده یک عملگر ریاضی است که ورودی‌ها را به خروجی مرتبط می‌کند.
   - این گراف محاسباتی نشان‌دهنده ساختار محاسباتی مورد نظر است و نحوه ترکیب و ارتباط بین تانسورها را مشخص می‌کند.
3. محاسبه مقدار خروجی:
   - پس از ایجاد گراف محاسباتی، می‌توان مقدار خروجی را محاسبه کرد.
   - این کار با اجرای عملیات ریاضی بر روی مقادیر ورودی‌ها و انتشار آن در گراف محاسباتی انجام می‌شود.
   - عملیات انتشار از ورودی‌ها به سمت خروجی، با استفاده از قوانین محاسبات ریاضی انجام می‌گیرد.
   - در هر گره گراف، عملیات مربوط به عملگر آن گره اجرا می‌شود و نتیجه به گره بعدی ارسال می‌گردد.
4. محاسبه مشتق‌ها (Backpropagation):
   - برای محاسبه مشتق‌ها، تابع backward() فراخوانی می‌شود.
   - ابتدا یک مرتب‌سازی توپولوژیکی از تانسورها انجام می‌شود تا ترتیب صحیح محاسبات مشخص شود.(در این مرحله، با استفاده از یک الگوریتم مرتب‌سازی توپولوژیکی، تنسورها به ترتیبی مرتب می‌شوند که هر تنسور قبل از تنسورهای وابسته به آن محاسبه شود.)
   - سپس از آخرین تانسور (خروجی) شروع کرده و به صورت معکوس (Reverse Mode) روی گراف محاسباتی حرکت می‌کند.
   - در این مرحله، قوانین زنجیره‌ای برای محاسبه مشتق‌ها به کار گرفته می‌شود.
5. محاسبه گرادیان در هر تانسور:
   - در هر تانسور، تابع _backward() فراخوانی می‌شود.
   - این تابع با استفاده از قوانین زنجیره‌ای، مشتق مقدار تانسور را نسبت به ورودی‌های خود محاسبه می‌کند.
   - نتیجه این محاسبات در ویژگی grad هر تانسور ذخیره می‌شود.
6. انتشار گرادیان‌ها در گراف:
   - به این ترتیب، گرادیان‌ها به صورت معکوس در گراف محاسباتی منتشر می‌شوند.
   - این انتشار گرادیان‌ها، مقادیر مشتق را نسبت به تمام پارامترهای شبکه (وزن‌ها و بایاس‌ها) محاسبه می‌کند.
   - این الگوریتم پس‌انتشار (Backpropagation)، امکان محاسبه مشتق‌گیری اتوماتیک را فراهم می‌کند.
   - این روش به طور گسترده در آموزش مدل‌های مبتنی بر گرادیان مانند شبکه‌های عصبی مصنوعی استفاده می‌شود.
</div>
</font>

In [ ]:
class Tensor:

  def __init__(self, value, children=()):
    self.value = value
    self.children = set(children)
    self.grad = 0
    self._backward = lambda  : None

  def __repr__(self) -> str:
    return f"Tensor(value={self.value})"

  def __mul__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)

    out = Tensor(self.value*other.value, children= (self, other))

    def backward():
      self.grad += other.value * out.grad
      other.grad += self.value * out.grad

    out._backward = backward

    return out

  def __add__(self, other):

    other = other if isinstance(other, Tensor) else Tensor(other)

    out = Tensor(self.value+other.value, children= (self, other))

    def backward():
      self.grad += 1 * out.grad
      other.grad += 1 * out.grad

    out._backward = backward

    return out

  def __radd__(self, other):
    return self + other

  def __pow__(self, other):

    input_value = self.value
    output_value = pow(input_value, other)

    out = Tensor(output_value, children=(self, ))

    def backward():
      self.grad += other * pow(self.value, other-1) * out.grad

    out._backward = backward

    return out

  def __sub__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)

    out = Tensor(self.value-other.value, children= (self, other))

    def backward():
      self.grad += 1 * out.grad
      other.grad += (-1) * out.grad

    out._backward = backward

    return out

  def backward(self):

    topo_sort = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v.children:
          build_topo(child)
        topo_sort.append(v)

    build_topo(self)

    self.grad = 1

    for node in reversed(topo_sort):
      node._backward()


<font face="B Mitra" size=4>
<div dir=rtl>
<p>
در کلاس F دو متد استاتیک برای توابع فعالساز تعریف می‌کنیم به طوری که تابع tanh یک تنسور می‌گیرد و یک تنسور که برابر است با مقدارtanh(x) است برمی‌گرداند که فرزند آن تنسور ورودی است. برای تابع backward هم مقدار گرادیان فرزند برابر است با مشتق tanh ضربدر گرادیان tanh

تابع sign هم پیاده سازی تابع علامت است که برای اعداد نامنفی تنسور ۱ و برای اعداد منفی تنسور -۱ را برمی‌گرداند.

</div>
</font>

In [ ]:
import math
class F:
  @staticmethod

  def tanh(X: Tensor) -> Tensor:
    input_value = X.value
    output_value = (math.exp(input_value) - math.exp(-input_value))/ (math.exp(input_value) + math.exp(-input_value))

    out = Tensor(output_value, children=(X, ))

    def backward():
      X.grad += (1-(output_value**2))*out.grad

    out._backward = backward

    return out

  def sign(x):
    if x.value >= 0:
      return Tensor(1)
    elif x.value < 0:
      return Tensor(-1)


<font face="B Mitra" size=4>
<div dir=rtl>
<p>
کلاس نورون در واقع کلاسی است که در آن مقدار ورودی‌های وارد به نورون در وزن متناظر انها ضرب می‌شود و با بایاس جمع می‌شود.
برای ساخت instance از کلاس neuron پارامتر اندازه ورودی گرفته می‌شود و در ابتدا وزن‌ها تنسوری های رندومی با مقادیر بین -۱ و ۱ است که اندازه لیست وزن‌ها برابر است با همان اندازه ورودی و بایاس هم عددی رندوم بین منفی یک و یک است.

متد forward برای ورودی x و وزن‌ها مقدار w_i * x_i متناظر را حساب می‌کند و جمع می‌بندد و تابع فعالساز را بر روی آن اعمال می‌کند و سپس برمی‌گرداند.

متد call هم صرفا متد forward را برای ورودی x صدا می‌زند

متد parameters در لیستی مقدار وزن‌ها و بایاس را برمی‌گرداند.

</div>
</font>

In [ ]:
import random

class Neuron:

  def __init__(self, input_size):

    self.weights = [Tensor(random.uniform(-1, 1)) for i in range(input_size)]
    self.bias = Tensor(random.uniform(-1, 1))

  def forward(self, x):
    res = sum([w_i * x_i for w_i, x_i in zip(self.weights, x)])
    return F.tanh(res + self.bias)

  def __call__(self, x):
    return self.forward(x)

  def parameters(self):
    return self.weights + [self.bias]



<font face="B Mitra" size=4>
<div dir=rtl>
<p>
کلاس Layer در واقع مجموعه ای از نمونه‌های کلاس Neuron است که لایه‌های شبکه   هستند و هر لایه باید بداند که تعداد نورون‌های ورودی چند تاست و به چه تعداد نورون باید تبدیل شود به همین دلیل در ساخت شی از پارامتر های input_size و output_size استفاده می‌کنیم و لیستی از نورون‌ها به تعداد سایز لایه خروجی با نورون‌هایی با سایز لایه ورودی را به عنوان اتریبیوت برای این کلاس در نظر می‌گیریم.

متد forward برای هر نورون لایه متد forward کلاس Neuron را کال می‌کند و به این شکل خروجی لایه را با اعمال بر روی هر نورون ورودی انجام  می‌دهد

متد call متد forward را صدا می‌زند برای ورودی x

متد parameters هم پارامترهای هر نورون که شامل وزن‌ها و بایاس‌ها است را بر می‌گرداند


</div>
</font>

In [ ]:
class Layer:

  def __init__(self, input_size, output_size):
    self.neurons = [Neuron(input_size) for _ in range(output_size)]

  def forward(self, x):
    out = [neuron(x) for neuron in self.neurons]
    return out[0] if len(out)==1 else out

  def __call__(self, x):
    return self.forward(x)

  def parameters(self):
    return [param for neuron in self.neurons for param in neuron.parameters()]



<font face="B Mitra" size=4>
<div dir=rtl>
<p>
کلاس MLP در واقع جایی است که شبکه تشکیل می‌شود به طوری که اندازه ورودی(تعداد فیچرها)و سایز لایه‌ها را می‌گیرد. لیست layers_total شامل تعداد نورون‌های هر لایه‌است که از کنار هم گذاشتن تعداد فیچرها و لیست تعداد نورون‌های هر لایه به وجود آمده است.

همانطور که گفته شد اشیای کلاس Layer با استفاده از تعداد نورون‌های ورودی و خروجی لایه ساخته می‌شوند به همین ترتیب لیستی از لایه‌ها را تشکیل می‌دهیم

متد forward برای هر لایه شبکه متد call لایه را صدا می‌زند و آن را بر می‌گرداند

و متد call هم متد forward را صدا می‌زند

متد parameters پارامترهای هر لایه را صدا می‌زند با استفاده از متد parametes در لایه ها.

</div>
</font>

In [ ]:
class MLP:

  def __init__(self, input_size, layer_sizes):
    layers_total = [input_size] + layer_sizes
    self.layers = [Layer(layers_total[i], layers_total[i+1]) for i in range(len(layer_sizes))]

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def __call__(self, x):
    return self.forward(x)

  def parameters(self):
    return [param for layer in self.layers for param in layer.parameters()]


<font face="B Mitra" size=4>
<div dir=rtl>
<p>
تابع criterion برای محاسبه خطا استفاده می‌شود به طوری که با گرفتن لیبل‌های
پیش بینی شده توسط شبکه و لیبل‌های درست جمع توان دو اختلاف لیبل‌ها را محاسبه می‌کند و در نهایت این مقدار بر تعداد رکوردها تقسیم می‌کند و برمی‌گرداند در واقع هدف شبکه کاهش مقدار همین تابع است.
</div>
</font>

In [ ]:
def criterion(y_hats: list[Tensor], Y) -> Tensor:
    loss = Tensor(0)
    for y_hat, y in zip(y_hats, Y):
        loss += pow(y_hat - y, 2)
    return pow(Tensor(len(y_hats)),-1)*loss


<font face="B Mitra" size=4>
<div dir=rtl>
<p>
کلاس optimizer برای ساخت نمونه، پارامترهای شبکه و نرخ یادگیری را می‌گیرد و به عنوان اتریبیوت ذخیره می‌کند
با استفاده از متد zero_grad در هر epoch مقدار گرادیان پارامترها را صفر می‌کنیم

با استفاده از متد step مقادیر را آپدیت میکنیم به طوری که مقدار جدید برابر است با مقدار فعلی منهای (گرادیان*نرخ یادگیری).

</div>
</font>

In [ ]:
class Optimizer:
    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def zero_grad(self):
        for param in self.parameters:
            param.grad = 0

    def step(self):
        for param in self.parameters:
            param.value -= self.lr * param.grad



---




<font face="B Mitra" size=4>
<div dir=rtl>
<p>
برای تست شبکه از مجموعه داده make_blobs کتابخانه sklearn استفاده می‌کنیم که دارای ۲۰ نمونه و ۱۵ فیچر است.لیبل‌های این مجموعه داده صفر یا یک هستند برای ایجاد تناسب با تابع فعالساز tanh لیبل های صفر را منفی یک تبدیل می‌کنیم.

</div>
</font>

In [ ]:
from sklearn.datasets import make_blobs
X, Y = make_blobs(n_samples=10, n_features=10, centers=2, random_state=41)
Y = [Tensor(-1) if i == 0 else Tensor(1) for i in Y]

<font face="B Mitra" size=4>
<div dir=rtl>
<p>
برای ساخت نمونه از مدل نیاز به اندازه ورودی داریم که همان تعداد فیچرهاست.و اندازه لایه‌های مخفی را به ترتیب ۲،۳ و لایه خروجی را ۱ نورون در نظر می‌گیریم و مدل را می‌سازیم
پارامترهای مدل ساخته شده و نرخ یادگیری را به عنوان ورودی برای ساخت شی optimizer استفاده می‌کنیم.
</div>
</font>

In [ ]:
input_size = X.shape[1]
layer_sizes = [2,3,1]
model = MLP(input_size, layer_sizes)
optimizer = Optimizer(model.parameters(), lr=0.005)

<font face="B Mitra" size=4>
<div dir=rtl>
<p>
در ادامه برای تعداد epoch = 20 y_hat را برای رکوردهای موجود در مجموعه داده بدست می‌آوریم و بر اساس تابع criterion مقدار loss که یک تنسور است بدست می‌آید سپس با استفاده از متد zero_grad گرادیان ها را صفر می‌کنیم و عملیات backward را برای loss شروع می‌کنیم و به همین ترتیب گرادیان ها مطابق بخش Tensor آپدیت می‌شوند سپس مقادیر بایاس و وزن ها را برحسب مشتقات آپدیت می‌کنیم

خروجی y_hats بین یک و منفی یک است با استفاده از تابع sign برای مقادیر نامنفی لیبل یک را اختصاص می‌دهیم و برای مقادیر منفی لیبل منفی یک را
</div>
</font>

In [ ]:
n_epochs = 200
for _ in range(n_epochs):
  sample = [x for x in X]
  y_hats = [model(x) for x in X]
  loss = criterion(y_hats, Y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

y_hats = [F.sign(i) for i in y_hats]
print((y_hats))
print(Y)

[Tensor(value=1), Tensor(value=1), Tensor(value=1), Tensor(value=1), Tensor(value=-1), Tensor(value=-1), Tensor(value=-1), Tensor(value=1), Tensor(value=-1), Tensor(value=-1)]
[Tensor(value=1), Tensor(value=1), Tensor(value=1), Tensor(value=1), Tensor(value=-1), Tensor(value=-1), Tensor(value=-1), Tensor(value=1), Tensor(value=-1), Tensor(value=-1)]
